> Betfair offers three API services:
1. **The Betting API** : contains navigation data, odds and volumes retrieval and bet placement operations
2. **The Exchange Stream API** : provides low latency access to market data allowing users to subscribe to and efficiently track changes to market, price and order data
3. **The Accounts API** : contains related operations such as the ability to retrieve your available account balance

Reference code for Betting and Accounts API's exists in python, for the exchange stream API: C#, Java & Node.js . This is an introductory .ipynb notebook to demonstrate python interaction with the Betting API for loading reference data and storing in a local database. Ultimately we would have several databases which store static information of use to us in live ordering systems and for historical analysis like : reference_data database, orders database, trades database etc.

In [1]:
import sqlite3
import pandas as pd
import requests
import json
from mysecrets import *
from ref_data import *

%load_ext autoreload
%autoreload 2

In [2]:
data_obj = ref_data(app_key,session_token)
data_obj.keep_alive()

SessionKey refresh: <Response [200]>


### Finding Event IDs

> An event is a single-day series of races at a single course

In [3]:
data_obj.find_events()
data_obj.events[:3]

[{'id': '31852918',
  'countryCode': 'gb',
  'venue': 'ayr',
  'openDate': '2022-10-24'},
 {'id': '31853015',
  'countryCode': 'gb',
  'venue': 'redcar',
  'openDate': '2022-10-24'},
 {'id': '31853013',
  'countryCode': 'gb',
  'venue': 'leicester',
  'openDate': '2022-10-24'}]

### SQL Databse insertion

> We have two tables in the info.db database, one for races (uniquely identified by event_id and market_id) and one for runners, uniquely identified by selection_id. We can add new instances to our existing info.db database using the existing class

In [4]:
data_obj.establish_database_connection('info.db')

connection established


### Adding races from events list

In [5]:
data_obj.add_races()

In [6]:
pd.read_sql_query("SELECT * FROM races", data_obj.connection).head()

,event_id,market_id,country,venue,name,date,time,number_runners,type,going,yards,seconds,total_vol,private_vol,pnl,order_count,bet_count
0,31852918,1.205409,gb,ayr,2m inhf,2022-10-24,15:25:00,10,None,None,None,None,None,None,None,None,None
1,31853015,1.205411,gb,redcar,6f hcap,2022-10-24,16:25:00,9,None,None,None,None,None,None,None,None,None
2,31853013,1.205411,gb,leicester,5f hcap,2022-10-24,15:35:00,10,None,None,None,None,None,None,None,None,None
3,31856050,1.205465,gb,bangor-on-dee,2m mdn nhf,2022-10-25,16:10:00,10,None,None,None,None,None,None,None,None,None
4,31855964,1.205463,gb,catterick,6f hcap,2022-10-25,16:30:00,9,None,None,None,None,None,None,None,None,None


### Adding Runners from Race listings

In [7]:
data_obj.add_runners()

In [8]:
pd.read_sql_query("SELECT * FROM runners", data_obj.connection).head()

,market_id,selection_id,runner_name,weight_lbs,jockey,individual_speed,win_lose,bsp,ppwap,ppmin,ppmax,pptradedvol
0,1.205409,10762561,big changes,None,None,None,None,None,None,None,None,None
1,1.205409,22071708,gipsy lee rose,None,None,None,None,None,None,None,None,None
2,1.205409,36259674,straight swap,None,None,None,None,None,None,None,None,None
3,1.205409,11830176,taxmeifyoucan,None,None,None,None,None,None,None,None,None
4,1.205409,38340860,camp belan,None,None,None,None,None,None,None,None,None


### Duplicate cleaning

> It's important to be able to remove duplicate additions if we re-run any methods accidentally. Also races are typically listed about two days in advance, so if you were to schedule an importer to run every day it would add duplicates. Let's re-add the runners as an example:

In [9]:
data_obj.add_runners()

Showing the existing duplicates:

In [10]:
pd.read_sql_query("SELECT * FROM runners WHERE EXISTS (SELECT 1 FROM runners p2 WHERE runners.market_id = p2.market_id AND runners.selection_id = p2.selection_id AND runners.rowid > p2.rowid);", data_obj.connection)

,market_id,selection_id,runner_name,weight_lbs,jockey,individual_speed,win_lose,bsp,ppwap,ppmin,ppmax,pptradedvol
0,1.205409,10762561,big changes,None,None,None,None,None,None,None,None,None
1,1.205409,22071708,gipsy lee rose,None,None,None,None,None,None,None,None,None
2,1.205409,36259674,straight swap,None,None,None,None,None,None,None,None,None
3,1.205409,11830176,taxmeifyoucan,None,None,None,None,None,None,None,None,None
4,1.205409,38340860,camp belan,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
638,1.205463,36404798,donnacha,None,None,None,None,None,None,None,None,None
639,1.205463,46846739,golden millie,None,None,None,None,None,None,None,None,None
640,1.205463,44049034,libor lad,None,None,None,None,None,None,None,None,None
641,1.205463,44169401,nathan walker,None,None,None,None,None,None,None,None,None


In [11]:
data_obj.delete_duplicates()

In [12]:
pd.read_sql_query("SELECT * FROM runners WHERE EXISTS (SELECT 1 FROM runners p2 WHERE runners.market_id = p2.market_id AND runners.selection_id = p2.selection_id AND runners.rowid > p2.rowid);", data_obj.connection)

,market_id,selection_id,runner_name,weight_lbs,jockey,individual_speed,win_lose,bsp,ppwap,ppmin,ppmax,pptradedvol


Now no duplicates, great